# simran4@wisc.edu, rgundavarapu@wisc.edu

## PART 1

In [1]:
#q1
from google.cloud import bigquery
bq = bigquery.Client(project="cs544-sp23-376003")
q1 = bq.query("""
SELECT geo_id
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'
LIMIT 5
""")
df = q1.to_dataframe()
(df['geo_id'][0])

'55025'

In [2]:
#q2
q2 = bq.query("""
SELECT COUNT(DISTINCT(county_name)) AS MEOW,state_fips_code
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY MEOW DESC
LIMIT 5
""")
df = q2.to_dataframe()
q2_dict={}
for i in df.iterrows():
    q2_dict[i[1].state_fips_code]=i[1].MEOW
q2_dict

{'48': 254, '13': 159, '51': 129, '21': 120, '29': 114}

In [3]:
#q3
no_cache = bigquery.QueryJobConfig(use_query_cache=False)
q1 = bq.query("""
SELECT geo_id
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'
LIMIT 5
""",job_config=no_cache)
q2 = bq.query("""
SELECT COUNT(DISTINCT(county_name)) AS MEOW,state_fips_code
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY MEOW DESC
LIMIT 5
""",job_config=no_cache)
q1cost=5 * (q1.total_bytes_billed / 1024**4)
q2cost=5 * (q2.total_bytes_billed / 1024**4)
q3_dict={}
q3_dict['q1']=q1cost
q3_dict['q2']=q2cost
q3_dict

{'q1': 4.76837158203125e-05, 'q2': 4.76837158203125e-05}

## PART 2

In [4]:
#q4
dataset = bigquery.Dataset("cs544-sp23-376003.p7")
bq.create_dataset(dataset, exists_ok=True)
[ds.dataset_id for ds in bq.list_datasets("cs544-sp23-376003")] # PASTE project name

['p7']

In [5]:
#q5
config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")
job = bq.load_table_from_uri(["gs://p7bucket/hdma-wi-2021.parquet"],
                             "cs544-sp23-376003.p7.loans", job_config=config)
q5 = bq.query("""
SELECT count(county_name) as county_count,county_name
FROM cs544-sp23-376003.p7.loans
INNER JOIN bigquery-public-data.geo_us_boundaries.counties 
ON counties.county_fips_code = loans.county_code
GROUP BY county_name
ORDER BY county_count DESC
LIMIT 10
""")
df = q5.to_dataframe()
q5_dict={}
for i in df.iterrows():
    q5_dict[i[1].county_name]=i[1].county_count
q5_dict

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

## PART 3

In [6]:
#q6
url = "https://drive.google.com/open?id=1e2qLPyxZ7s5ibMyEg7bxX2wWYAQD2ROAUcZuv8fqhnA"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table(dataset.table('apps'))
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)
q6 = bq.query("""
SELECT COUNT(income) as count
FROM p7.apps
WHERE INCOME = 150000
LIMIT 10
""")
df = q6.to_dataframe()
## MY CHOSEN INCOME WAS 150K
df['count'][0]

8

In [7]:
#q7
from geopandas import GeoDataFrame
q71 = bq.query("""
CREATE OR REPLACE TABLE cs544-sp23-376003.p7.houses
AS
SELECT *, ST_GEOGPOINT(longitude, latitude) AS loc
FROM p7.apps
""")
q = bq.query("""
CREATE OR REPLACE TABLE cs544-sp23-376003.p7.wi
AS

SELECT county_name, county_geom
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE state_fips_code = '55' 
""")
q72=bq.query("""
SELECT  county_name , COUNT(county_name)
FROM p7.houses, cs544-sp23-376003.p7.wi
WHERE ST_WITHIN(houses.loc, wi.county_geom) 
GROUP BY county_name
""")
houses_df = q72.to_dataframe()
q7_dict={}
for i in houses_df.iterrows():
    q7_dict[i[1].county_name]=i[1].f0_
q7_dict

{'Door': 5,
 'Brown': 3,
 'Dane': 31,
 'Kewaunee': 1,
 'La Crosse': 1,
 'Dodge': 1,
 'Jefferson': 4,
 'Bayfield': 3,
 'Walworth': 2,
 'Ashland': 1,
 'Milwaukee': 2,
 'Barron': 1,
 'Oneida': 1,
 'Waukesha': 1,
 'Iron': 1,
 'Sauk': 1,
 'Monroe': 1,
 'Adams': 1,
 'Columbia': 2,
 'Sheboygan': 1,
 'Green Lake': 1,
 'Iowa': 1}

## PART 4

In [8]:
#q8
q8 = bq.query("""
CREATE OR REPLACE MODEL `cs544-sp23-376003.p7.mymodel`
OPTIONS(model_type='LINEAR_REG', INPUT_LABEL_COLS=['loan_amount'])
AS
SELECT income, loan_term, loan_amount
FROM `cs544-sp23-376003.p7.loans`
""")
q82=bq.query("""
SELECT *
FROM
ML.EVALUATE(
    MODEL `cs544-sp23-376003.p7.mymodel`,
    (
        SELECT income, loan_term, loan_amount
        FROM `cs544-sp23-376003.p7.loans`
    )
)

""")
df=q82.to_dataframe()
df['r2_score'][0]

0.29165412288020354

In [9]:
#q9
q9=bq.query("""
SELECT * FROM
ML.WEIGHTS(MODEL `cs544-sp23-376003.p7.mymodel`)
""")
df=q9.to_dataframe()
df['weight'][0]

0.8057773037176653

In [10]:
#q10
q10=bq.query("""
SELECT *
FROM
ML.PREDICT(
    MODEL `cs544-sp23-376003.p7.mymodel`,
    (
        SELECT income, 360 AS loan_term, loan_amount
        FROM `p7.apps`
    )
)
""")
df=q10.to_dataframe()
counter=0
for i in df.iterrows():
    if i[1].predicted_loan_amount < i[1].loan_amount:
        counter+=1
counter/len(df)

0.6016949152542372